In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.utils import shuffle
from sklearn.svm import SVR


In [6]:
!wget https://raw.githubusercontent.com/venzera/KIBAI/tm.csv

--2024-10-07 09:26:28--  https://raw.githubusercontent.com/venzera/KIBAI/tm.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 400 Bad Request
2024-10-07 09:26:28 ERROR 400: Bad Request.



In [7]:
df = pd.read_csv('tm.csv')
df = df[df['pH'] == 7]

Результаты с 1 семинара: [тут](https://github.com/venzera/KIBAI/blob/main/seminar1.py)



**Linear Regression** - MSE: 149.43571159581018, R2: 0.10525302750615484


**Random Forest** - MSE: 66.79733975293495, R2:  0.60

In [8]:
df

,seq_id,protein_sequence,pH,data_source,tm
0,0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,doi.org/10.1038/s41592-020-0801-4,75.7
1,1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,doi.org/10.1038/s41592-020-0801-4,50.5
2,2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,doi.org/10.1038/s41592-020-0801-4,40.5
3,3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,doi.org/10.1038/s41592-020-0801-4,47.2
4,4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,doi.org/10.1038/s41592-020-0801-4,49.5
...,...,...,...,...,...
31385,31385,YYMYSGGGSALAAGGGGAGRKGDWNDIDSIKKKDLHHSRGDEKAQG...,7.0,doi.org/10.1038/s41592-020-0801-4,51.8
31386,31386,YYNDQHRLSSYSVETAMFLSWERAIVKPGAMFKKAVIGFNCNVDLI...,7.0,doi.org/10.1038/s41592-020-0801-4,37.2
31387,31387,YYQRTLGAELLYKISFGEMPKSAQDSAENCPSGMQFPDTAIAHANV...,7.0,doi.org/10.1038/s41592-020-0801-4,64.6
31388,31388,YYSFSDNITTVFLSRQAIDDDHSLSLGTISDVVESENGVVAADDAR...,7.0,doi.org/10.1038/s41592-020-0801-4,50.7


In [9]:
def getOneHot(seq):
  amino_acids = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
  #amino_acid_dict = {aa: 0 for aa in amino_acids}
  amino_acid_dict = {}
  for i in amino_acids:
    amino_acid_dict[i]  = 0

  for aa in seq:
    amino_acid_dict[aa] += 1
  #dict = {key: value}
  onehot = list(amino_acid_dict.values())
  #print(amino_acid_dict.keys())
  #print(amino_acid_dict.values())
  return onehot

In [10]:


def custom_train_test_split(X, y, test_size=0.1, random_state=None):

    if random_state is not None:
        np.random.seed(random_state)
    X = np.asarray(X)
    y = np.asarray(y)
    sorted_indices = np.argsort(y)
    X_sorted = X[sorted_indices]
    y_sorted = y[sorted_indices]
    n_samples = len(y)
    n_test = int(n_samples * test_size)
    step = n_samples // n_test

    test_indices = np.arange(0, n_samples, step)[:n_test]

    test_mask = np.zeros(n_samples, dtype=bool)
    test_mask[test_indices] = True
    train_mask = ~test_mask

    X_test, y_test = X_sorted[test_mask], y_sorted[test_mask]
    X_train, y_train = X_sorted[train_mask], y_sorted[train_mask]

    X_train, y_train = shuffle(X_train, y_train, random_state=random_state)
    X_test, y_test = shuffle(X_test, y_test, random_state=random_state)

    return X_train, X_test, y_train, y_test

In [11]:
df['onehot'] = df['protein_sequence'].apply(getOneHot)

In [12]:
df

,seq_id,protein_sequence,pH,data_source,tm,onehot
0,0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,doi.org/10.1038/s41592-020-0801-4,75.7,"[45, 1, 13, 30, 13, 38, 3, 14, 16, 37, 8, 5, 1..."
1,1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,doi.org/10.1038/s41592-020-0801-4,50.5,"[28, 0, 10, 52, 6, 18, 4, 13, 19, 23, 2, 6, 8,..."
2,2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,doi.org/10.1038/s41592-020-0801-4,40.5,"[50, 9, 27, 32, 21, 65, 11, 16, 39, 18, 6, 15,..."
3,3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,doi.org/10.1038/s41592-020-0801-4,47.2,"[20, 5, 19, 29, 12, 16, 7, 10, 17, 28, 2, 9, 1..."
4,4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,doi.org/10.1038/s41592-020-0801-4,49.5,"[86, 14, 78, 78, 32, 84, 40, 71, 68, 104, 31, ..."
...,...,...,...,...,...,...
31385,31385,YYMYSGGGSALAAGGGGAGRKGDWNDIDSIKKKDLHHSRGDEKAQG...,7.0,doi.org/10.1038/s41592-020-0801-4,51.8,"[33, 12, 38, 31, 18, 51, 15, 21, 32, 46, 13, 2..."
31386,31386,YYNDQHRLSSYSVETAMFLSWERAIVKPGAMFKKAVIGFNCNVDLI...,7.0,doi.org/10.1038/s41592-020-0801-4,37.2,"[37, 5, 21, 29, 22, 27, 22, 30, 20, 47, 14, 19..."
31387,31387,YYQRTLGAELLYKISFGEMPKSAQDSAENCPSGMQFPDTAIAHANV...,7.0,doi.org/10.1038/s41592-020-0801-4,64.6,"[13, 1, 7, 7, 7, 11, 2, 6, 8, 6, 7, 5, 6, 8, 3..."
31388,31388,YYSFSDNITTVFLSRQAIDDDHSLSLGTISDVVESENGVVAADDAR...,7.0,doi.org/10.1038/s41592-020-0801-4,50.7,"[47, 5, 34, 36, 23, 52, 11, 34, 35, 45, 26, 25..."


In [13]:
hydrophobicity = {'A': 1.8, 'R': -4.5, 'N': -3.5, 'D': -3.5, 'C': 2.5, 'Q': -3.5, 'E': -3.5, 'G': -0.4,
                  'H': -3.2, 'I': 4.5, 'L': 3.8, 'K': -3.9, 'M': 1.9, 'F': 2.8, 'P': -1.6, 'S': -0.8,
                  'T': -0.7, 'W': -0.9, 'Y': -1.3, 'V': 4.2}

molecular_weight = {'A': 89.1, 'R': 174.2, 'N': 132.1, 'D': 133.1, 'C': 121.2, 'Q': 146.2, 'E': 147.1, 'G': 75.1,
                    'H': 155.2, 'I': 131.2, 'L': 131.2, 'K': 146.2, 'M': 149.2, 'F': 165.2, 'P': 115.1, 'S': 105.1,
                    'T': 119.1, 'W': 204.2, 'Y': 181.2, 'V': 117.1}

charge = {'A': 0, 'R': 1, 'N': 0, 'D': -1, 'C': 0, 'Q': 0, 'E': -1, 'G': 0,
          'H': 0, 'I': 0, 'L': 0, 'K': 1, 'M': 0, 'F': 0, 'P': 0, 'S': 0,
          'T': 0, 'W': 0, 'Y': 0, 'V': 0}



1. Amino acid counts (20 features)
2. Amino acid percentages (20 features)
3. Sequence length (1 feature)
4. Average hydrophobicity (1 feature)
5. Average molecular weight (1 feature)
6. Net charge (1 feature)

In [14]:
def getEnrichedEncoding(seq):
    amino_acids = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
    aa_count = {aa: seq.count(aa) for aa in amino_acids}
    seq_length = len(seq)
    aa_percentages = {aa: count / seq_length * 100 for aa, count in aa_count.items()}

    avg_hydrophobicity = sum(hydrophobicity[aa] * count for aa, count in aa_count.items()) / seq_length
    avg_molecular_weight = sum(molecular_weight[aa] * count for aa, count in aa_count.items()) / seq_length
    net_charge = sum(charge[aa] * count for aa, count in aa_count.items())
    encoding = (
        list(aa_count.values()) +
        list(aa_percentages.values()) +
        [seq_length, avg_hydrophobicity, avg_molecular_weight, net_charge]
    )

    return encoding

In [15]:
df['enriched_encoding'] = df['protein_sequence'].apply(getEnrichedEncoding)

In [16]:
len(df['enriched_encoding'].iloc[1])

44

In [17]:
df

,seq_id,protein_sequence,pH,data_source,tm,onehot,enriched_encoding
0,0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,doi.org/10.1038/s41592-020-0801-4,75.7,"[45, 1, 13, 30, 13, 38, 3, 14, 16, 37, 8, 5, 1...","[45, 1, 13, 30, 13, 38, 3, 14, 16, 37, 8, 5, 1..."
1,1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,doi.org/10.1038/s41592-020-0801-4,50.5,"[28, 0, 10, 52, 6, 18, 4, 13, 19, 23, 2, 6, 8,...","[28, 0, 10, 52, 6, 18, 4, 13, 19, 23, 2, 6, 8,..."
2,2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,doi.org/10.1038/s41592-020-0801-4,40.5,"[50, 9, 27, 32, 21, 65, 11, 16, 39, 18, 6, 15,...","[50, 9, 27, 32, 21, 65, 11, 16, 39, 18, 6, 15,..."
3,3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,doi.org/10.1038/s41592-020-0801-4,47.2,"[20, 5, 19, 29, 12, 16, 7, 10, 17, 28, 2, 9, 1...","[20, 5, 19, 29, 12, 16, 7, 10, 17, 28, 2, 9, 1..."
4,4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,doi.org/10.1038/s41592-020-0801-4,49.5,"[86, 14, 78, 78, 32, 84, 40, 71, 68, 104, 31, ...","[86, 14, 78, 78, 32, 84, 40, 71, 68, 104, 31, ..."
...,...,...,...,...,...,...,...
31385,31385,YYMYSGGGSALAAGGGGAGRKGDWNDIDSIKKKDLHHSRGDEKAQG...,7.0,doi.org/10.1038/s41592-020-0801-4,51.8,"[33, 12, 38, 31, 18, 51, 15, 21, 32, 46, 13, 2...","[33, 12, 38, 31, 18, 51, 15, 21, 32, 46, 13, 2..."
31386,31386,YYNDQHRLSSYSVETAMFLSWERAIVKPGAMFKKAVIGFNCNVDLI...,7.0,doi.org/10.1038/s41592-020-0801-4,37.2,"[37, 5, 21, 29, 22, 27, 22, 30, 20, 47, 14, 19...","[37, 5, 21, 29, 22, 27, 22, 30, 20, 47, 14, 19..."
31387,31387,YYQRTLGAELLYKISFGEMPKSAQDSAENCPSGMQFPDTAIAHANV...,7.0,doi.org/10.1038/s41592-020-0801-4,64.6,"[13, 1, 7, 7, 7, 11, 2, 6, 8, 6, 7, 5, 6, 8, 3...","[13, 1, 7, 7, 7, 11, 2, 6, 8, 6, 7, 5, 6, 8, 3..."
31388,31388,YYSFSDNITTVFLSRQAIDDDHSLSLGTISDVVESENGVVAADDAR...,7.0,doi.org/10.1038/s41592-020-0801-4,50.7,"[47, 5, 34, 36, 23, 52, 11, 34, 35, 45, 26, 25...","[47, 5, 34, 36, 23, 52, 11, 34, 35, 45, 26, 25..."


In [18]:
X = df['onehot'].to_list()
y = df['tm'].to_list()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify = None, random_state=42)

In [19]:
X = df['onehot'].to_list()
y = df['tm'].to_list()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify = None, random_state=42)

RGReg_model = RandomForestRegressor(n_estimators=100, random_state=42).fit(X_train, y_train)

y_pred_rf = RGReg_model.predict(X_test)

print(mean_squared_error(y_test, y_pred_rf), r2_score(y_test, y_pred_rf))


66.79733975293495 0.6000506379878139


In [ ]:
X = df['onehot'].to_list()
y = df['tm'].to_list()
X_train, X_test, y_train, y_test = custom_train_test_split(X, y, test_size=0.1, random_state=42)

RGReg_model = RandomForestRegressor(n_estimators=100, random_state=42).fit(X_train, y_train)

y_pred_rf = RGReg_model.predict(X_test)

print(mean_squared_error(y_test, y_pred_rf), r2_score(y_test, y_pred_rf))


In [ ]:
X = df['enriched_encoding'].to_list()
y = df['tm'].to_list()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify = None, random_state=42)

RGReg_model = RandomForestRegressor(n_estimators=100, random_state=42).fit(X_train, y_train)

y_pred_rf = RGReg_model.predict(X_test)

print(mean_squared_error(y_test, y_pred_rf), r2_score(y_test, y_pred_rf))


In [ ]:
X = df['enriched_encoding'].to_list()
y = df['tm'].to_list()
X_train, X_test, y_train, y_test = custom_train_test_split(X, y, test_size=0.1, random_state=42)

RGReg_model = RandomForestRegressor(n_estimators=100, random_state=42).fit(X_train, y_train)

y_pred_rf = RGReg_model.predict(X_test)

print(mean_squared_error(y_test, y_pred_rf), r2_score(y_test, y_pred_rf))
